In [ ]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.metrics.pairwise import cosine_similarity

host = "svc-c73ce0ea-2976-4fec-8231-9b8c8bbdf115-dml.aws-virginia-8.svc.singlestore.com"
port = 3306
user = "admin"
password = "y42sJU2U6sdKYKZgU9kEhDiFvTenJhlv"
database = "openaidatabase"

connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

try:

    resumes_query = """
        SELECT names AS resume_id, resume_embeddings, years_of_experience AS resume_experience, 
               major, skills 
        FROM resumes_data;
    """
    jd_query = """
        SELECT company_name, required_major, skills AS jd_skills, 
               years_of_experience AS jd_experience, jd_embeddings 
        FROM JD_data;
    """
    
    resumes_df = pd.read_sql(resumes_query, con=engine)
    jd_df = pd.read_sql(jd_query, con=engine)

    jd_df['company_id'] = jd_df.index + 1

    resumes_df['resume_embeddings'] = resumes_df['resume_embeddings'].apply(lambda x: np.frombuffer(x, dtype=np.float32))
    jd_df['jd_embeddings'] = jd_df['jd_embeddings'].apply(lambda x: np.frombuffer(x, dtype=np.float32))


    major_keywords = {"computer science", "statistics", "economics", "data science"}

    all_results = []


    weights = {
        "jd_embedding": 0.90,  
        "skills": 0.05,
        "major": 0.05
    }

    for jd_index, jd_row in jd_df.iterrows():
        jd_results = []
        jd_skills = set(jd_row['jd_skills'].lower().split(","))

        for resume_index, resume_row in resumes_df.iterrows():
            if jd_row['jd_experience'] <= resume_row['resume_experience']:

                jd_similarity = cosine_similarity([resume_row['resume_embeddings']], [jd_row['jd_embeddings']])[0][0]

                resume_skills = set(resume_row['skills'].lower().split(","))
                skills_match = len(jd_skills.intersection(resume_skills)) / len(jd_skills) if len(jd_skills) > 0 else 0.0

                jd_major_keywords = set(jd_row['required_major'].lower().split(",")).intersection(major_keywords)
                resume_major_keywords = set(resume_row['major'].lower().split(",")).intersection(major_keywords)
                major_match = len(jd_major_keywords.intersection(resume_major_keywords)) / len(jd_major_keywords) if len(jd_major_keywords) > 0 else 0.0

                final_score = (
                    jd_similarity * weights["jd_embedding"] +
                    skills_match * weights["skills"] +
                    major_match * weights["major"]
                )

                jd_results.append({
                    "company_id": jd_row['company_id'],
                    "resume_id": resume_row['resume_id'],
                    "jd_similarity_score": jd_similarity,
                    "skills_score": skills_match,
                    "major_score": major_match,
                    "final_score": final_score
                })

        jd_results = sorted(jd_results, key=lambda x: x['final_score'], reverse=True)[:50]

        for rank, result in enumerate(jd_results, start=1):
            result['rank'] = rank
            all_results.append(result)

    final_results_df = pd.DataFrame(all_results)


    output_path = "jd_resume_major_keywords_weighted_results.csv"
    final_results_df.to_csv(output_path, index=False)
    print(f"匹配结果已保存为 {output_path} 文件")

except Exception as e:
    print("Error occurred:", e)

finally:
    engine.dispose()


ImportError: dlopen(/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-38-darwin.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib
  Referenced from: <BEFB6E07-597A-3758-A60B-141E8E215EF0> /opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-38-darwin.so
  Reason: tried: '/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/_isolve/liblapack.3.dylib' (no such file), '/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/_isolve/../../../../../../liblapack.3.dylib' (no such file), '/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/_isolve/liblapack.3.dylib' (no such file), '/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/_isolve/../../../../../../liblapack.3.dylib' (no such file), '/opt/anaconda3/bin/../lib/liblapack.3.dylib' (no such file), '/opt/anaconda3/bin/../lib/liblapack.3.dylib' (no such file), '/usr/local/lib/liblapack.3.dylib' (no such file), '/usr